### Model 6 : YOLOv11 Combined Training


In [1]:
import os
import shutil
import yaml
from pathlib import Path
from ultralytics import YOLO


In [2]:

class CombinedDatasetTrainer:
    def __init__(self):
        self.base_dir = r"D:\Salma\mmm\Homework3"
        self.dataset_paths = {
            'moroccan': os.path.join(self.base_dir, 'Moroccan_Pothole_Dataset-main', 'YOLO', 'Original'),
            'potholev1': os.path.join(self.base_dir, 'Pothole.v1-raw.yolov11')
        }
        self.combined_dir = os.path.join(self.base_dir, 'Combined_Dataset')
        
    def create_combined_dataset(self):
        print("Creating combined dataset structure...")
        
        for split in ['train', 'valid']:
            for subdir in ['images', 'labels']:
                os.makedirs(os.path.join(self.combined_dir, split, subdir), exist_ok=True)
        
        counts = {'train': 0, 'valid': 0}
        
        for dataset_name, dataset_path in self.dataset_paths.items():
            print(f"\nProcessing {dataset_name} dataset...")
            
            for split in ['train', 'valid']:
                src_split = split if os.path.exists(os.path.join(dataset_path, split)) else 'val'
                
                src_img_dir = os.path.join(dataset_path, src_split, 'images')
                src_label_dir = os.path.join(dataset_path, src_split, 'labels')
                tgt_img_dir = os.path.join(self.combined_dir, split, 'images')
                tgt_label_dir = os.path.join(self.combined_dir, split, 'labels')
                
                if os.path.exists(src_img_dir):
                    for img_file in os.listdir(src_img_dir):
                        if img_file.endswith(('.jpg', '.jpeg', '.png')):
                            new_name = f"{dataset_name}_{split}_{img_file}"
                            label_file = img_file.rsplit('.', 1)[0] + '.txt'
                            new_label_name = f"{dataset_name}_{split}_{label_file}"
                            
                            shutil.copy2(
                                os.path.join(src_img_dir, img_file),
                                os.path.join(tgt_img_dir, new_name)
                            )
                            
                            if os.path.exists(os.path.join(src_label_dir, label_file)):
                                shutil.copy2(
                                    os.path.join(src_label_dir, label_file),
                                    os.path.join(tgt_label_dir, new_label_name)
                                )
                            counts[split] += 1
        
        yaml_path = os.path.join(self.combined_dir, 'data.yaml')
        yaml_content = {
            'path': self.combined_dir,
            'train': 'train/images',
            'val': 'valid/images',
            'names': ['pothole'],
            'nc': 1
        }
        
        with open(yaml_path, 'w') as f:
            yaml.dump(yaml_content, f)
        
        print("\nDataset Statistics:")
        print(f"Training images: {counts['train']}")
        print(f"Validation images: {counts['valid']}")
        
        return yaml_path
    
    def train(self, yaml_path):
        try:
            print("\nInitializing YOLOv11 model...")
            model = YOLO('yolo11n.pt')
            
            print("\nStarting training with enhanced augmentation...")
            results = model.train(
                data=yaml_path,
                epochs=150,                # Increased epochs
                imgsz=[480, 640, 800],     # Multi-scale training
                batch=8,                   # Adjusted batch size
                
                # Training parameters
                lr0=0.01,
                lrf=0.001,
                momentum=0.937,
                weight_decay=0.0005,
                warmup_epochs=10,          # Extended warmup
                cos_lr=True,              # Cosine learning rate
                
                # Enhanced augmentation
                mosaic=1.0,               # Mosaic augmentation
                mixup=0.3,                # Image mixup
                scale=0.7,                # Random scaling
                degrees=45.0,             # Random rotation
                translate=0.3,            # Translation
                shear=10.0,               # Shear transformation
                perspective=0.001,         # Perspective transform
                flipud=0.5,               # Vertical flip
                fliplr=0.5,               # Horizontal flip
                hsv_h=0.015,              # HSV-Hue
                hsv_s=0.7,                # HSV-Saturation
                hsv_v=0.4,                # HSV-Value
                copy_paste=0.3,           # Copy-paste augmentation
                
                save=True,
                name='combined_training_enhanced'
            )
            
            return model, results
            
        except Exception as e:
            print(f"Error during training: {str(e)}")
            raise
    
    def evaluate_on_moroccan(self, model):
        try:
            print("\nEvaluating model on Moroccan validation set...")
            
            moroccan_yaml = os.path.join(self.dataset_paths['moroccan'], 'moroccan_val.yaml')
            yaml_content = {
                'path': self.dataset_paths['moroccan'],
                'train': 'train/images',
                'val': 'valid/images',
                'names': ['pothole'],
                'nc': 1
            }
            
            with open(moroccan_yaml, 'w') as f:
                yaml.dump(yaml_content, f)
            
            try:
                metrics = model.val(data=moroccan_yaml)
                
                precision = float(metrics.box.p[0])  # Fixed array indexing
                recall = float(metrics.box.r[0])
                map50 = float(metrics.box.map50[0])
                
                f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
                
                print("\nResults on Moroccan Validation Set:")
                print(f"Precision: {precision:.3f}")
                print(f"Recall: {recall:.3f}")
                print(f"F1 Score: {f1_score:.3f}")
                print(f"mAP@0.5: {map50:.3f}")
                
                print("\nComparison with baseline:")
                print(f"Current F1 Score: {f1_score:.3f} (Baseline: 0.72)")
                print(f"Current mAP@0.5: {map50:.3f} (Baseline: 0.74)")
                
                return metrics
                
            finally:
                if os.path.exists(moroccan_yaml):
                    os.remove(moroccan_yaml)
                    
        except Exception as e:
            print(f"Error during evaluation: {str(e)}")
            return None


In [3]:

def main():
    try:
        trainer = CombinedDatasetTrainer()
        yaml_path = trainer.create_combined_dataset()
        model, results = trainer.train(yaml_path)
        metrics = trainer.evaluate_on_moroccan(model)
        model.save('pothole_enhanced_final.pt')
        print("\nTraining completed successfully!")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Creating combined dataset structure...

Processing moroccan dataset...

Processing potholev1 dataset...

Dataset Statistics:
Training images: 519
Validation images: 148

Initializing YOLOv11 model...

Starting training with enhanced augmentation...
New https://pypi.org/project/ultralytics/8.3.40 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.39  Python-3.10.11 torch-2.1.2+cpu CPU (12th Gen Intel Core(TM) i7-12700H)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=D:\Salma\mmm\Homework3\Combined_Dataset\data.yaml, epochs=150, time=None, patience=100, batch=8, imgsz=[480, 640, 800], save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=combined_training_enhanced3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_rati

train: Scanning D:\Salma\mmm\Homework3\Combined_Dataset\train\labels.cache... 519 images, 0 backgrounds, 0 corrupt: 100%|██████████| 519/519 [00:00<?, ?it/s]
val: Scanning D:\Salma\mmm\Homework3\Combined_Dataset\valid\labels.cache... 148 images, 0 backgrounds, 0 corrupt: 100%|██████████| 148/148 [00:00<?, ?it/s]


Plotting labels to runs\detect\combined_training_enhanced3\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 800 train, 800 val
Using 0 dataloader workers
Logging results to runs\detect\combined_training_enhanced3
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/150         0G      2.316       3.29      2.276         64        800: 100%|██████████| 65/65 [03:50<00:00,  3.55s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:23<00:00,  2.31s/it]

                   all        148        381    0.00597       0.48      0.137     0.0384


...


150 epochs completed in 9.165 hours.
Optimizer stripped from runs\detect\combined_training_enhanced3\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\combined_training_enhanced3\weights\best.pt, 5.5MB

Validating runs\detect\combined_training_enhanced3\weights\best.pt...
Ultralytics 8.3.39  Python-3.10.11 torch-2.1.2+cpu CPU (12th Gen Intel Core(TM) i7-12700H)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:14<00:00,  1.42s/it]


                   all        148        381      0.746      0.588      0.673      0.354
Speed: 1.4ms preprocess, 84.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\combined_training_enhanced3

Evaluating model on Moroccan validation set...
Ultralytics 8.3.39  Python-3.10.11 torch-2.1.2+cpu CPU (12th Gen Intel Core(TM) i7-12700H)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning D:\Salma\mmm\Homework3\Moroccan_Pothole_Dataset-main\YOLO\Original\valid\labels.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.03it/s]


                   all         15         51      0.674      0.314      0.384      0.113
Speed: 1.5ms preprocess, 108.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\combined_training_enhanced32
Error during evaluation: invalid index to scalar variable.

Training completed successfully!
